In [1]:
import datetime
import sqlite3 as sql 
import tweepy
import re
import pandas as pd

In [1]:
def add_query(twitter_query, con):
    cur = con.cursor()
    time = datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ")
    cur.execute(f"""
        INSERT INTO queries (query, First_ran) VALUES ('{twitter_query}','{time}')
    """)
    con.commit()
    con.close()


    
def get_queried_tweets(con, client, querydf_row, run_time):
    cur = con.cursor()
    query_idx = querydf_row['query_idx']
    query = querydf_row['query']
    if querydf_row['Last_ran'] is None:
        start_time = querydf_row['First_ran']
    else:
        start_time = querydf_row['Last_ran']
    res = client.search_recent_tweets(query=query, start_time=start_time ,expansions=["author_id"] )
    cur.execute(f"""UPDATE queries
                    SET Last_ran = '{run_time}'
                    WHERE query_idx = {query_idx}""")
    con.commit()
    
    for i in range(len(res.includes['users'])):
        try:
            user_id = res.includes['users'][i].id
            public_name = res.includes['users'][i].name
            user_name = res.includes['users'][i].username
            tweet_id = res.data[i].id
            tweet_text = res.data[i].text
            tweet_text =tweet_text.replace('\n', ' ').replace('"',"'")
            retweet = False
            rt_user = None
            retweet_str = re.findall(r"RT @\S+", tweet_text)
            if retweet_str:
                retweet = True 
                rt_user = re.findall(r"@\S+",retweet_str[0].replace(":",""))
            users = re.findall(r"@\S+", tweet_text)
            if retweet_str:
                rtu = retweet_str[0].replace('RT ','')
                users.remove(rtu)
            urls = re.findall(r'https?:\S+', tweet_text)
            hash_tags = re.findall(r'#\S+', tweet_text)
            sql_query_text = f'''INSERT INTO tweets (user_id, tweet_id, query_idx, urls, text, users, retweet, rt_user, hash_tags, pull_time) 
            VALUES ({user_id}, {tweet_id}, {query_idx}, "{urls}", "{tweet_text}", "{users}", "{retweet}", "{rt_user}", "{hash_tags}", "{run_time}")'''
            sql_query_user = f'''INSERT INTO users (user_id , name , username)
            VALUES ("{user_id}","{public_name}","{user_name}")
            ON CONFLICT(user_id) DO UPDATE SET capture_count=capture_count+1
            '''
        
            cur.execute(sql_query_user)
            cur.execute(sql_query_text)
            con.commit()
        except Exception as e:
            print(e)

    

def init_scrap(con, client, query=None):
    output_date = datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ")
    if query:
        df = pd.read_sql_query(f"SELECT * FROM queries WHERE query_idx = {query}",con)   
    else:
        df = pd.read_sql_query("SELECT * FROM queries",con)   
    for idx, row in df.iterrows():
        get_queried_tweets(con, client, row, output_date)
    con.close()